In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/WiDSWorldWide_GlobalDathon26/sample_submission.csv
/kaggle/input/WiDSWorldWide_GlobalDathon26/train.csv
/kaggle/input/WiDSWorldWide_GlobalDathon26/metaData.csv
/kaggle/input/WiDSWorldWide_GlobalDathon26/test.csv


In [2]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import KFold
from scipy.stats import norm

In [3]:
train = pd.read_csv('/kaggle/input/WiDSWorldWide_GlobalDathon26/train.csv')
test = pd.read_csv('/kaggle/input/WiDSWorldWide_GlobalDathon26/test.csv')

In [4]:
ID_COL = "event_id"
TIME_COL = "time_to_hit_hours"
EVENT_COL = "event"

features = [c for c in train.columns 
            if c not in [ID_COL, TIME_COL, EVENT_COL]]

X_test = test[features]

In [5]:
# =====================================================
# STEP 1 — Reuse Model-4 Core (Survival + Binary)
# =====================================================

aft_params = {
    "objective": "survival:aft",
    "eval_metric": "aft-nloglik",
    "aft_loss_distribution": "normal",
    "aft_loss_distribution_scale": 2.1,
    "learning_rate": 0.03,
    "max_depth": 4,
    "subsample": 0.85,
    "colsample_bytree": 0.85,
    "min_child_weight": 4,
    "seed": 42
}

binary_params = {
    "objective": "binary:logistic",
    "eval_metric": "logloss",
    "learning_rate": 0.05,
    "max_depth": 3,
    "subsample": 0.9,
    "colsample_bytree": 0.9,
    "seed": 42
}

kf = KFold(n_splits=5, shuffle=True, random_state=42)

# ---------- Survival ----------
surv_preds = []

for tr_idx, _ in kf.split(train):
    X_tr = train.iloc[tr_idx][features]
    y_time = train.iloc[tr_idx][TIME_COL].values
    y_event = train.iloc[tr_idx][EVENT_COL].values

    lower = y_time
    upper = y_time.copy()
    upper[y_event == 0] = np.inf

    dtrain = xgb.DMatrix(X_tr)
    dtrain.set_float_info("label_lower_bound", lower)
    dtrain.set_float_info("label_upper_bound", upper)

    dtest = xgb.DMatrix(X_test)
    model = xgb.train(aft_params, dtrain, num_boost_round=700)
    surv_preds.append(model.predict(dtest))

mu = np.mean(surv_preds, axis=0)
sigma = aft_params["aft_loss_distribution_scale"]

def surv_prob(h):
    return norm.cdf((np.log(h) - mu) / sigma)

# ---------- Binary ----------
def binary_target(h):
    return ((train[TIME_COL] <= h) & (train[EVENT_COL] == 1)).astype(int)

def train_binary(h):
    preds = []
    y = binary_target(h)

    for tr_idx, _ in kf.split(train):
        dtrain = xgb.DMatrix(train.iloc[tr_idx][features], label=y.iloc[tr_idx])
        dtest = xgb.DMatrix(X_test)
        model = xgb.train(binary_params, dtrain, num_boost_round=400)
        preds.append(model.predict(dtest))

    return np.mean(preds, axis=0)

# raw probabilities
p12 = 0.5 * train_binary(12) + 0.5 * surv_prob(12)
p24 = 0.55 * train_binary(24) + 0.45 * surv_prob(24)
p48 = 0.70 * train_binary(48) + 0.30 * surv_prob(48)
p72 = 0.65 * train_binary(72) + 0.35 * surv_prob(72)

# =====================================================
# STEP 2 — Horizon-wise Temperature Scaling (NEW)
# =====================================================

def temperature_scale(p, T):
    eps = 1e-6
    p = np.clip(p, eps, 1 - eps)
    logit = np.log(p / (1 - p))
    return 1 / (1 + np.exp(-logit / T))

# tuned temperatures (safe, not aggressive)
p12 = temperature_scale(p12, T=1.15)
p24 = temperature_scale(p24, T=1.10)
p48 = temperature_scale(p48, T=1.05)  # minimal change, preserve best horizon
p72 = temperature_scale(p72, T=1.08)

# =====================================================
# STEP 3 — Mid-Probability Lift (Anti-underconfidence)
# =====================================================

def mid_lift(p, strength=0.04):
    return p + strength * p * (1 - p)

p12 = mid_lift(p12, 0.04)
p24 = mid_lift(p24, 0.04)
p48 = mid_lift(p48, 0.03)
p72 = mid_lift(p72, 0.03)

# =====================================================
# STEP 4 — Final Smoothing + Monotonicity
# =====================================================

global_rate = train[EVENT_COL].mean()

def smooth(p):
    return 0.95 * p + 0.05 * global_rate

p12 = smooth(p12)
p24 = smooth(p24)
p48 = smooth(p48)
p72 = smooth(p72)

probs = np.vstack([p12, p24, p48, p72]).T
probs = np.maximum.accumulate(probs, axis=1)

p12, p24, p48, p72 = probs.T

# =====================================================
# SUBMISSION
# =====================================================

submission = pd.DataFrame({
    "event_id": test[ID_COL],
    "prob_12h": np.clip(p12, 0, 1),
    "prob_24h": np.clip(p24, 0, 1),
    "prob_48h": np.clip(p48, 0, 1),
    "prob_72h": np.clip(p72, 0, 1),
})

submission.to_csv("submission_v5.csv", index=False)
submission.head()

,event_id,prob_12h,prob_24h,prob_48h,prob_72h
0,10662602,0.019911,0.021465,0.021465,0.024028
1,13353600,0.303533,0.580684,0.723830,0.723830
2,13942327,0.018794,0.019952,0.019952,0.023969
3,16112781,0.389035,0.509739,0.678251,0.678251
4,17132808,0.026509,0.026509,0.026509,0.026509


In [6]:
print(len(submission))

95
